In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely.ops import nearest_points

import seaborn as sns

from mpl_toolkits.axes_grid1 import make_axes_locatable

import math

import time

from matplotlib import cm

import matplotlib.lines as mlines

%matplotlib inline

## Monitoring Data

In [2]:
df = pd.read_csv('EDF_Data.csv', header = 1)
df.tail()

,Longitude,Latitude,NO Value,NO2 Value,BC Value
21483,-122.034943,37.560076,129.999995,44.778220,3.923761
21484,-122.034724,37.560164,60.799998,39.027545,1.408693
21485,-122.034681,37.559830,34.622951,28.816797,2.659885
21486,-122.034504,37.559958,74.764705,35.735434,1.776353
21487,-122.034503,37.559957,78.754782,41.062757,2.014664


### Split into BC and NO2

In [3]:
BC_df = df[['Longitude', 'Latitude', 'BC Value']]

In [4]:
NO2_df = df[['Longitude', 'Latitude', 'NO2 Value']]

## Facility Level Data with Source Type

In [5]:
Facility_PM = pd.read_csv("Data/PM_Facility_Data.csv")

In [6]:
Facility_PM.drop(columns = ['Unnamed: 0'], inplace=True)

In [7]:
Facility_NO2 = pd.read_csv("Data/NO2_Facility_Data.csv")

In [8]:
Facility_NO2.drop(columns = ['Unnamed: 0'], inplace=True)

## Calculate distance from monitoring data to each Facility - PM

In [9]:
Facility_PM_All = Facility_PM.copy()

In [10]:
Facility_PM.head()

,eis facility id,company,source-category,latitude,longitude,city,PM25,PM10,units,source-type,PM25-transform,ems-type
0,126911,PACIFIC-BELL,Institution,37.76685,-122.24972,ALAMEDA,0.000731,0.000749,TON,Retail-Res,-7.220508,medium
1,127111,PACIFIC-BELL,Institution,37.86771,-122.26784,BERKELEY,0.002173,0.002226,TON,Retail-Res,-6.131671,medium
2,127511,PACIFIC-BELL,Institution,37.80603,-122.26946,OAKLAND,0.008291,0.008494,TON,Retail-Res,-4.792633,medium
3,127611,PACIFIC-BELL,Institution,37.78422,-122.22241,OAKLAND,0.000706,0.000724,TON,Retail-Res,-7.255446,medium
4,127711,PACIFIC-BELL,Institution,37.83336,-122.26307,OAKLAND,0.001554,0.001592,TON,Retail-Res,-6.466989,medium


In [11]:
### Create a columns as eis-source-type
Facility_PM['eis-source'] =    Facility_PM['eis facility id'].apply(str) + '-' +  Facility_PM['source-type']   

In [12]:
### Create a columns as eis-source-ems-type
Facility_PM['eis-source-ems'] =    Facility_PM['eis-source']   + '-' + Facility_PM['ems-type']

In [13]:
Facility_PM.head(100)

,eis facility id,company,source-category,latitude,longitude,city,PM25,PM10,units,source-type,PM25-transform,ems-type,eis-source,eis-source-ems
0,126911,PACIFIC-BELL,Institution,37.76685,-122.24972,ALAMEDA,0.000731,0.000749,TON,Retail-Res,-7.220508,medium,126911-Retail-Res,126911-Retail-Res-medium
1,127111,PACIFIC-BELL,Institution,37.86771,-122.26784,BERKELEY,0.002173,0.002226,TON,Retail-Res,-6.131671,medium,127111-Retail-Res,127111-Retail-Res-medium
2,127511,PACIFIC-BELL,Institution,37.80603,-122.26946,OAKLAND,0.008291,0.008494,TON,Retail-Res,-4.792633,medium,127511-Retail-Res,127511-Retail-Res-medium
3,127611,PACIFIC-BELL,Institution,37.78422,-122.22241,OAKLAND,0.000706,0.000724,TON,Retail-Res,-7.255446,medium,127611-Retail-Res,127611-Retail-Res-medium
4,127711,PACIFIC-BELL,Institution,37.83336,-122.26307,OAKLAND,0.001554,0.001592,TON,Retail-Res,-6.466989,medium,127711-Retail-Res,127711-Retail-Res-medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,10471211,CITY-OF-ALAMEDA-/-PUBLIC-WORKS,Institution,37.78543,-122.28292,ALAMEDA,0.000053,0.000054,TON,Retail-Res,-9.847996,low,10471211-Retail-Res,10471211-Retail-Res-low
96,10472111,VERIZON-WIRELESS-815053-(HW,Institution,37.75152,-122.19886,OAKLAND,0.000118,0.000121,TON,Retail-Res,-9.046268,low,10472111-Retail-Res,10472111-Retail-Res-low
97,10472311,VERIZON-WIRELESS-EAST-14TH/H,Institution,37.76697,-122.21838,OAKLAND,0.000166,0.000170,TON,Retail-Res,-8.701254,medium,10472311-Retail-Res,10472311-Retail-Res-medium
98,10472411,VERIZON-WIRELESS-(ALAMEDA-PERM,Institution,37.79890,-122.28399,OAKLAND,0.000060,0.000062,TON,Retail-Res,-9.714108,low,10472411-Retail-Res,10472411-Retail-Res-low


In [14]:
### Add an empty column for distance
Facility_PM['dist'] = 0
Facility_PM['dist'].astype(float)

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
343    0.0
344    0.0
345    0.0
346    0.0
347    0.0
Name: dist, Length: 348, dtype: float64

### Create a dataframe where each column is one of the industrial facilities. Add one column for latitude, longitude and PM2.5 emissions value. Dropping PM10 emissions since Black Carbon is a form on PM2.5 emissions

In [15]:
# Create individual dataframes
Oak_PM_lat = Facility_PM[['eis-source-ems', 'latitude']]
Oak_PM_long = Facility_PM[['eis-source-ems', 'longitude']]
Oak_PM_PM25 = Facility_PM[['eis-source-ems', 'PM25']]
Oak_PM_dist = Facility_PM[['eis-source-ems', 'dist']]


In [16]:
# Transpose all the dataframes
Oak_PM_lat = Oak_PM_lat.T
Oak_PM_long = Oak_PM_long.T
Oak_PM_PM25 = Oak_PM_PM25.T
Oak_PM_dist = Oak_PM_dist.T




In [17]:
## Make the header as the first row in each transposed dataframe
Oak_PM_lat = Oak_PM_lat.rename(columns=Oak_PM_lat.iloc[0]).drop(Oak_PM_lat.index[0])
Oak_PM_long = Oak_PM_long.rename(columns=Oak_PM_long.iloc[0]).drop(Oak_PM_long.index[0])
Oak_PM_PM25 = Oak_PM_PM25.rename(columns=Oak_PM_PM25.iloc[0]).drop(Oak_PM_PM25.index[0])
Oak_PM_dist = Oak_PM_dist.rename(columns=Oak_PM_dist.iloc[0]).drop(Oak_PM_dist.index[0])

#Oak_PM_PM10 = Oak_PM_PM10.rename(columns=Oak_PM_PM10.iloc[0].astype(int)).drop(Oak_PM_PM10.index[0])
#Oak_PM_emsdist = Oak_PM_emsdist.rename(columns=Oak_PM_emsdist.iloc[0]).drop(Oak_PM_emsdist.index[0])
#Oak_PM_geo = Oak_PM_geo.rename(columns=Oak_PM_geo.iloc[0]).drop(Oak_PM_geo.index[0])

In [18]:
## Add suffix to column header based on the dataframe type
Oak_PM_lat.columns = [str(col) + '_latitude' for col in Oak_PM_lat.columns]
Oak_PM_long.columns = [str(col) + '_longitude' for col in Oak_PM_long.columns]
Oak_PM_PM25.columns = [str(col) + '_PM25' for col in Oak_PM_PM25.columns]
Oak_PM_dist.columns = [str(col) + '_dist' for col in Oak_PM_dist.columns]


#Oak_PM_PM10.columns = [str(col) + '_PM10' for col in Oak_PM_PM10.columns]
#Oak_PM_emsdist.columns = [str(col) + '_emsdist' for col in Oak_PM_emsdist.columns]
#Oak_PM_geo.columns = [str(col) + '_geo' for col in Oak_PM_geo.columns]

In [19]:
## Remove index for each dataframe
Oak_PM_lat.reset_index(drop=True, inplace=True)
Oak_PM_long.reset_index(drop=True, inplace=True)
Oak_PM_PM25.reset_index(drop=True, inplace=True)
Oak_PM_dist.reset_index(drop=True, inplace=True)

#Oak_PM_PM10.reset_index(drop=True, inplace=True)
#Oak_PM_emsdist.reset_index(drop=True, inplace=True)
#Oak_PM_geo.reset_index(drop=True, inplace=True)

In [20]:
### Combine individual dataframes into one
Oak_PM_combined = Oak_PM_lat.join(Oak_PM_long).join(Oak_PM_PM25).join(Oak_PM_dist)





In [21]:
### Sort based on column names
Oak_PM_combined = Oak_PM_combined.reindex(columns=sorted(Oak_PM_combined.columns))

In [22]:
Oak_PM_combined

,10452611-Manufacturing-low_PM25,10452611-Manufacturing-low_dist,10452611-Manufacturing-low_latitude,10452611-Manufacturing-low_longitude,10452911-Retail-Res-high_PM25,10452911-Retail-Res-high_dist,10452911-Retail-Res-high_latitude,10452911-Retail-Res-high_longitude,10453011-WWTP-medium_PM25,10453011-WWTP-medium_dist,...,804911-FoodPlant-medium_latitude,804911-FoodPlant-medium_longitude,808611-AsphaltPlant-high_PM25,808611-AsphaltPlant-high_dist,808611-AsphaltPlant-high_latitude,808611-AsphaltPlant-high_longitude,808811-FoodPlant-high_PM25,808811-FoodPlant-high_dist,808811-FoodPlant-high_latitude,808811-FoodPlant-high_longitude
0,4e-09,0,37.8133,-122.268,0.051645,0,37.7457,-122.235,0.000361007,0,...,37.6971,-122.173,2.15679,0,37.8698,-122.302,0.590966,0,37.7936,-122.268


In [23]:
#Create a datafram where each row contains emissions of PM2.5 for each facility
Oak_PM_combined = Oak_PM_combined.loc[Oak_PM_combined.index.repeat(21488)].reset_index(drop=True)

In [24]:
combined_BC_Facility = BC_df.join(Oak_PM_combined)

In [25]:
combined_BC_Facility.head()

,Longitude,Latitude,BC Value,10452611-Manufacturing-low_PM25,10452611-Manufacturing-low_dist,10452611-Manufacturing-low_latitude,10452611-Manufacturing-low_longitude,10452911-Retail-Res-high_PM25,10452911-Retail-Res-high_dist,10452911-Retail-Res-high_latitude,...,804911-FoodPlant-medium_latitude,804911-FoodPlant-medium_longitude,808611-AsphaltPlant-high_PM25,808611-AsphaltPlant-high_dist,808611-AsphaltPlant-high_latitude,808611-AsphaltPlant-high_longitude,808811-FoodPlant-high_PM25,808811-FoodPlant-high_dist,808811-FoodPlant-high_latitude,808811-FoodPlant-high_longitude
0,-122.322594,37.806781,0.818032,4e-09,0,37.8133,-122.268,0.051645,0,37.7457,...,37.6971,-122.173,2.15679,0,37.8698,-122.302,0.590966,0,37.7936,-122.268
1,-122.322310,37.806150,0.551475,4e-09,0,37.8133,-122.268,0.051645,0,37.7457,...,37.6971,-122.173,2.15679,0,37.8698,-122.302,0.590966,0,37.7936,-122.268
2,-122.322301,37.806420,0.593712,4e-09,0,37.8133,-122.268,0.051645,0,37.7457,...,37.6971,-122.173,2.15679,0,37.8698,-122.302,0.590966,0,37.7936,-122.268
3,-122.322299,37.805880,0.489898,4e-09,0,37.8133,-122.268,0.051645,0,37.7457,...,37.6971,-122.173,2.15679,0,37.8698,-122.302,0.590966,0,37.7936,-122.268
4,-122.322267,37.806689,0.739341,4e-09,0,37.8133,-122.268,0.051645,0,37.7457,...,37.6971,-122.173,2.15679,0,37.8698,-122.302,0.590966,0,37.7936,-122.268


In [26]:
# Convert distance or emissions distance column to float type
for idx, col in enumerate(combined_BC_Facility.columns):
        if "_dist" in col:
            combined_BC_Facility[col] = pd.to_numeric(combined_BC_Facility[col], downcast="float")



KeyboardInterrupt: 

## Calculate distance between point of measurement and each facility and add it to the '_dist' column

In [53]:
### Defining a function to calculate the distance between two GPS coordinates (latitude and longitude)
def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d


In [284]:
time1 = time.time()
for index, row in combined_BC_Facility.iterrows():
    for idx, col in enumerate(combined_BC_Facility.columns):
        if "_dist" in col:
            combined_BC_Facility.at[index,col] = float(distance((row.iloc[1], row.iloc[0]), (row.iloc[idx+1], row.iloc[idx+2])))*0.621
            #BC_Facility.at[index,col] = float(row.iloc[idx])
time2 = time.time()            
    
print(time2 - time1)

274.76193618774414


In [285]:
### Write this to a dataframe
combined_BC_Facility.to_csv("Data/Combined_BC_Dist.csv")



In [286]:
combined_BC_Facility.shape

(21488, 1395)

## Calculate distance from monitoring data to each Facility - NO2

In [35]:
Facility_NO2_All = Facility_NO2.copy()

In [36]:
Facility_NO2.head()

,eis facility id,company name,source-category,latitude,longitude,city,NO2,source-type,NO2-transform,ems-type
0,126911,PACIFIC-BELL,Institution,37.76685,-122.24972,ALAMEDA,0.054659,Retail-Res,-2.906636,medium
1,127111,PACIFIC-BELL,Institution,37.86771,-122.26784,BERKELEY,0.162383,Retail-Res,-1.817800,medium
2,127511,PACIFIC-BELL,Institution,37.80603,-122.26946,OAKLAND,0.619550,Retail-Res,-0.478762,high
3,127611,PACIFIC-BELL,Institution,37.78422,-122.22241,OAKLAND,0.052783,Retail-Res,-2.941574,medium
4,127711,PACIFIC-BELL,Institution,37.83336,-122.26307,OAKLAND,0.116122,Retail-Res,-2.153117,medium


In [37]:
### Create a columns as eis-source-type
Facility_NO2['eis-source'] =    Facility_NO2['eis facility id'].apply(str) + '-' +  Facility_NO2['source-type']   

In [38]:
### Create a columns as eis-source-ems-type
Facility_NO2['eis-source-ems'] =    Facility_NO2['eis-source']   + '-' + Facility_NO2['ems-type']

In [39]:
### Add an empty column for distance
Facility_NO2['dist'] = 0
Facility_NO2['dist'].astype(float)

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
312    0.0
313    0.0
314    0.0
315    0.0
316    0.0
Name: dist, Length: 317, dtype: float64

In [40]:
# Create individual dataframes
Oak_NO2_lat = Facility_NO2[['eis-source-ems', 'latitude']]
Oak_NO2_long = Facility_NO2[['eis-source-ems', 'longitude']]
Oak_NO2_NO2 = Facility_NO2[['eis-source-ems', 'NO2']]
Oak_NO2_dist = Facility_NO2[['eis-source-ems', 'dist']]


In [41]:
# Transpose all the dataframes
Oak_NO2_lat = Oak_NO2_lat.T
Oak_NO2_long = Oak_NO2_long.T
Oak_NO2_NO2 = Oak_NO2_NO2.T
Oak_NO2_dist = Oak_NO2_dist.T


In [42]:
## Make the header as the first row in each transposed dataframe
Oak_NO2_lat = Oak_NO2_lat.rename(columns=Oak_NO2_lat.iloc[0]).drop(Oak_NO2_lat.index[0])
Oak_NO2_long = Oak_NO2_long.rename(columns=Oak_NO2_long.iloc[0]).drop(Oak_NO2_long.index[0])
Oak_NO2_NO2 = Oak_NO2_NO2.rename(columns=Oak_NO2_NO2.iloc[0]).drop(Oak_NO2_NO2.index[0])
Oak_NO2_dist = Oak_NO2_dist.rename(columns=Oak_NO2_dist.iloc[0]).drop(Oak_NO2_dist.index[0])


In [43]:
## Add suffix to column header based on the dataframe type
Oak_NO2_lat.columns = [str(col) + '_latitude' for col in Oak_NO2_lat.columns]
Oak_NO2_long.columns = [str(col) + '_longitude' for col in Oak_NO2_long.columns]
Oak_NO2_NO2.columns = [str(col) + '_NO2' for col in Oak_NO2_NO2.columns]
Oak_NO2_dist.columns = [str(col) + '_dist' for col in Oak_NO2_dist.columns]


In [44]:
## Remove index for each dataframe
Oak_NO2_lat.reset_index(drop=True, inplace=True)
Oak_NO2_long.reset_index(drop=True, inplace=True)
Oak_NO2_NO2.reset_index(drop=True, inplace=True)
Oak_NO2_dist.reset_index(drop=True, inplace=True)

In [45]:
### Combine individual dataframes into one
Oak_NO2_combined = Oak_NO2_lat.join(Oak_NO2_long).join(Oak_NO2_NO2).join(Oak_NO2_dist)

In [46]:
### Sort based on column names
Oak_NO2_combined = Oak_NO2_combined.reindex(columns=sorted(Oak_NO2_combined.columns))

In [47]:
Oak_NO2_combined

,10452911-Retail-Res-high_NO2,10452911-Retail-Res-high_dist,10452911-Retail-Res-high_latitude,10452911-Retail-Res-high_longitude,10453011-WWTP-medium_NO2,10453011-WWTP-medium_dist,10453011-WWTP-medium_latitude,10453011-WWTP-medium_longitude,10457011-Manufacturing-medium_NO2,10457011-Manufacturing-medium_dist,...,804911-FoodPlant-high_latitude,804911-FoodPlant-high_longitude,808611-AsphaltPlant-high_NO2,808611-AsphaltPlant-high_dist,808611-AsphaltPlant-high_latitude,808611-AsphaltPlant-high_longitude,808811-FoodPlant-high_NO2,808811-FoodPlant-high_dist,808811-FoodPlant-high_latitude,808811-FoodPlant-high_longitude
0,0.629648,0,37.7457,-122.235,0.0269778,0,37.7587,-122.214,0.0461694,0,...,37.6971,-122.173,1.7035,0,37.8698,-122.302,0.27833,0,37.7936,-122.268


In [48]:
#Create a datafram where each row contains emissions of PM2.5 for each facility
Oak_NO2_combined = Oak_NO2_combined.loc[Oak_NO2_combined.index.repeat(21488)].reset_index(drop=True)

In [49]:
combined_NO2_Facility = NO2_df.join(Oak_NO2_combined)

In [50]:
combined_NO2_Facility.head()

,Longitude,Latitude,NO2 Value,10452911-Retail-Res-high_NO2,10452911-Retail-Res-high_dist,10452911-Retail-Res-high_latitude,10452911-Retail-Res-high_longitude,10453011-WWTP-medium_NO2,10453011-WWTP-medium_dist,10453011-WWTP-medium_latitude,...,804911-FoodPlant-high_latitude,804911-FoodPlant-high_longitude,808611-AsphaltPlant-high_NO2,808611-AsphaltPlant-high_dist,808611-AsphaltPlant-high_latitude,808611-AsphaltPlant-high_longitude,808811-FoodPlant-high_NO2,808811-FoodPlant-high_dist,808811-FoodPlant-high_latitude,808811-FoodPlant-high_longitude
0,-122.322594,37.806781,17.539762,0.629648,0,37.7457,-122.235,0.0269778,0,37.7587,...,37.6971,-122.173,1.7035,0,37.8698,-122.302,0.27833,0,37.7936,-122.268
1,-122.322310,37.806150,19.956750,0.629648,0,37.7457,-122.235,0.0269778,0,37.7587,...,37.6971,-122.173,1.7035,0,37.8698,-122.302,0.27833,0,37.7936,-122.268
2,-122.322301,37.806420,23.967768,0.629648,0,37.7457,-122.235,0.0269778,0,37.7587,...,37.6971,-122.173,1.7035,0,37.8698,-122.302,0.27833,0,37.7936,-122.268
3,-122.322299,37.805880,18.435184,0.629648,0,37.7457,-122.235,0.0269778,0,37.7587,...,37.6971,-122.173,1.7035,0,37.8698,-122.302,0.27833,0,37.7936,-122.268
4,-122.322267,37.806689,25.797037,0.629648,0,37.7457,-122.235,0.0269778,0,37.7587,...,37.6971,-122.173,1.7035,0,37.8698,-122.302,0.27833,0,37.7936,-122.268


In [51]:
# Convert distance or emissions distance column to float type
for idx, col in enumerate(combined_NO2_Facility.columns):
        if "_dist" in col:
            combined_NO2_Facility[col] = pd.to_numeric(combined_NO2_Facility[col], downcast="float")



In [54]:
time1 = time.time()
for index, row in combined_NO2_Facility.iterrows():
    for idx, col in enumerate(combined_NO2_Facility.columns):
        if "_dist" in col:
            combined_NO2_Facility.at[index,col] = float(distance((row.iloc[1], row.iloc[0]), (row.iloc[idx+1], row.iloc[idx+2])))*0.621
            #BC_Facility.at[index,col] = float(row.iloc[idx])
time2 = time.time()            
    
print(time2 - time1)

243.63282704353333


In [55]:
### Write this to a dataframe
combined_NO2_Facility.to_csv("Data/Combined_NO2_Dist.csv")

